In [ ]:
# Cell 1: imports & config
import os
from pathlib import Path
import time
import threading
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

import cv2
import schedule
import smtplib
from email.message import EmailMessage

# load environment variables
load_dotenv()
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
CAREGIVER_EMAIL = os.getenv("CAREGIVER_EMAIL")


In [ ]:
# Cell 3: data generators
DATA_DIR = Path("data")  # expect data/train and data/val inside
IMG_SIZE = (224,224)
BATCH = 16

train_dir = str(DATA_DIR / "train")
val_dir = str(DATA_DIR / "val")

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=IMG_SIZE,
                                              batch_size=BATCH,
                                              class_mode='categorical')

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=IMG_SIZE,
                                          batch_size=BATCH,
                                          class_mode='categorical')
class_indices = train_gen.class_indices
print("Classes:", class_indices)


In [ ]:
# Cell 4: model
base = MobileNetV2(input_shape=IMG_SIZE+(3,), include_top=False, weights='imagenet')
base.trainable = False  # freeze base for initial training

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(class_indices), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# Cell 5: train
EPOCHS = 10
history = model.fit(train_gen,
                    epochs=EPOCHS,
                    validation_data=val_gen)

# Save model
model.save("pill_classifier_mobilenetv2.h5")


In [ ]:
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, epochs=5, validation_data=val_gen)
model.save("pill_classifier_mobilenetv2_ft.h5")


In [ ]:
# Cell 6: load & predict util
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model("pill_classifier_mobilenetv2.h5")  # or ft model

inv_class_map = {v:k for k,v in class_indices.items()}

def predict_image(img_bgr):
    # img_bgr: numpy array BGR (OpenCV)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, IMG_SIZE)
    x = img_resized.astype("float32") / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    idx = np.argmax(preds[0])
    conf = preds[0][idx]
    label = inv_class_map[idx]
    return label, float(conf)
